In [1]:
from google.colab import userdata
AGENT_OPS_KEY=userdata.get('AGENT_OPS_KEY')

In [2]:
import agentops

from autogen import ConversableAgent, UserProxyAgent, config_list_from_json

agentops.init(api_key=AGENT_OPS_KEY)

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)
🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=b03ea012-5ea5-4fd7-8603-8045cf5e7bd0


In [3]:
GROQ_API_KEY=userdata.get('GROQ_API_KEY')
config_list = [
    {
        # Let's choose the Llama 3 model
        "model": "llama-3.1-8b-instant",
        # Put your Groq API key here or put it into the GROQ_API_KEY environment variable.
        "api_key": GROQ_API_KEY,
        # We specify the API Type as 'groq' so it uses the Groq client class
        "api_type": "groq",
    }
]

In [4]:
import agentops

# When initializing AgentOps, you can pass in optional tags to help filter sessions
agentops.init(tags=["autogen-website"])

# Create the agent that uses the LLM.
assistant = ConversableAgent("agent", llm_config={"config_list": config_list})

# Create the agent that represents the user in the conversation.
user_proxy = UserProxyAgent("user", code_execution_config=False)

# Let the assistant start the conversation.  It will end when the user types "exit".
assistant.initiate_chat(user_proxy, message="How can I help you today?")

# Close your AgentOps session to indicate that it completed.
agentops.end_session("Success")

🖇 AgentOps: AgentOps has already been initialized. If you are trying to start a session, call agentops.start_session() instead.


agent (to user):

How can I help you today?

--------------------------------------------------------------------------------
Replying as user. Provide feedback to agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation: 4*6
user (to agent):

4*6

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
agent (to user):

The result of 4 multiplied by 6 is 24.


/usr/local/lib/python3.10/dist-packages/autogen/oai/groq.py:280: UserWarning: Cost calculation not available for model llama-3.1-8b-instant
  warnings.warn(f"Cost calculation not available for model {model}", UserWarning)



--------------------------------------------------------------------------------
Replying as user. Provide feedback to agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation: exit


🖇 AgentOps: This run's cost $0.00
🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=b03ea012-5ea5-4fd7-8603-8045cf5e7bd0


In [5]:
from typing import Annotated, Literal

from autogen import ConversableAgent, config_list_from_json, register_function
import google.generativeai as genai
from google.generativeai.types import HarmCategory, HarmBlockThreshold

agentops.start_session(tags=["autogen-tool-example"])

Operator = Literal["+", "-", "*", "/"]


def calculator(a: int, b: int, operator: Annotated[Operator, "operator"]) -> int:
    if operator == "+":
        return a + b
    elif operator == "-":
        return a - b
    elif operator == "*":
        return a * b
    elif operator == "/":
        return int(a / b)
    else:
        raise ValueError("Invalid operator")


# Create the agent that uses the LLM.
assistant = ConversableAgent(
    name="Assistant",
    system_message="You are a helpful AI assistant. "
    "You can help with simple calculations. "
    "Return 'TERMINATE' when the task is done.",
    llm_config={"config_list": config_list},
)

# The user proxy agent is used for interacting with the assistant agent
# and executes tool calls.
user_proxy = ConversableAgent(
    name="User",
    llm_config=False,
    is_termination_msg=lambda msg: msg.get("content") is not None and "TERMINATE" in msg["content"],
    human_input_mode="NEVER",
)

assistant.register_for_llm(name="calculator", description="A simple calculator")(calculator)
user_proxy.register_for_execution(name="calculator")(calculator)

# Register the calculator function to the two agents.
register_function(
    calculator,
    caller=assistant,  # The assistant agent can suggest calls to the calculator.
    executor=user_proxy,  # The user proxy agent can execute the calculator calls.
    name="calculator",  # By default, the function name is used as the tool name.
    description="A simple calculator",  # A description of the tool.
)

# Let the assistant start the conversation.  It will end when the user types "exit".
user_proxy.initiate_chat(assistant, message="What is (1423 - 123) / 3 + (32 + 23) * 5?")

agentops.end_session("Success")

🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=112024e9-ac91-49fb-961d-85aa30c97c65


User (to Assistant):

What is (1423 - 123) / 3 + (32 + 23) * 5?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...


/usr/local/lib/python3.10/dist-packages/autogen/agentchat/conversable_agent.py:2567: UserWarning: Function 'calculator' is being overridden.
  warnings.warn(f"Function '{tool_sig['function']['name']}' is being overridden.", UserWarning)
/usr/local/lib/python3.10/dist-packages/autogen/agentchat/conversable_agent.py:2486: UserWarning: Function 'calculator' is being overridden.
  warnings.warn(f"Function '{name}' is being overridden.", UserWarning)


RuntimeError: Groq exception occurred: Error code: 400 - {'error': {'message': "Failed to call a function. Please adjust your prompt. See 'failed_generation' for more details.", 'type': 'invalid_request_error', 'code': 'tool_use_failed', 'failed_generation': '<function=calculator>{"a": 1423, "b": 123, "operator": "-"}</function>\n<function=calculator>{"a": 1423 - 123, "b": 3, "operator": "/"}</function>\n<function=calculator>{"a": 32, "b": 23, "operator": "+"}</function>\n<function=calculator>{"a": 32 + 23, "b": 5, "operator": "*"></function>\n<function=calculator>{"a": (32 + 23) * 5, "b": 3, "operator": "+"}</function>\nTERMINATE'}}